In [1]:
%showtypes on

Types will be printed.


In [2]:
spark

Waiting for a Spark session to start...

Spark application_1540458187951_76634: Some(http://gw02.itversity.com:4045)

In [3]:
sc.getConf.getAll.filter(_._2.contains("/proxy/"))(0)._2

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1540458187951_76634

In [4]:
def getType(o: Any) = o.getClass.getCanonicalName

getType: (o: Any)String


In [5]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getenv("HOME").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [6]:
val path = hdfs_home+"/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet/"

path = /user/kranthidr/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet/


/user/kranthidr/dataSets/spark-guide/flight-data/parquet/2010-summary.parquet/

In [7]:
case class Flight(DEST_COUNTRY_NAME: String,
                  ORIGIN_COUNTRY_NAME: String, 
                  count: BigInt)

defined class Flight


In [8]:
val flightsDF = spark.
                read.
                parquet(path)

flightsDF = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [9]:
val flights = flightsDF.as[Flight].cache()

flights = [DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]


[DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field]

In [10]:
println(flightsDF.getClass().getName)
println(flights.getClass().getName)

org.apache.spark.sql.Dataset
org.apache.spark.sql.Dataset


In [11]:
flights.show(2)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|    1|
|    United States|            Ireland|  264|
+-----------------+-------------------+-----+
only showing top 2 rows



In [12]:
flights.first.DEST_COUNTRY_NAME

United States

In [13]:
def originIsDestination(flight_row: Flight): Boolean = {
  return flight_row.ORIGIN_COUNTRY_NAME == flight_row.DEST_COUNTRY_NAME
}

originIsDestination: (flight_row: Flight)Boolean


In [14]:
flights.filter(x => originIsDestination(x)).collect()

[Flight(United States,United States,348113)]

In [15]:
flights.collect().filter(x => originIsDestination(x))

[Flight(United States,United States,348113)]

In [16]:
flights.filter(x => originIsDestination(x)).show()

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|348113|
+-----------------+-------------------+------+



In [17]:
val destinations = flights.map(f => f.DEST_COUNTRY_NAME)

destinations = [value: string]


[value: string]

In [18]:
val localDestinations = destinations.take(5)

localDestinations = Array(United States, United States, United States, Egypt, Equatorial Guinea)


[United States, United States, United States, Egypt, Equatorial Guinea]

In [19]:
case class FlightMetadata(count: BigInt, randomData: BigInt)

defined class FlightMetadata


In [20]:
spark.range(500).map(x => (x, scala.util.Random.nextLong)).show()

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  0|-5183701258119263509|
|  1| 8983338770324918353|
|  2| 1547762808549480808|
|  3| 7091183739042959552|
|  4|-2441987337371666469|
|  5|-4090101999647053880|
|  6| -994483537529894480|
|  7|-7163532810552512961|
|  8|-7836866877273270709|
|  9|-7817897955762684866|
| 10| 7728398141736455413|
| 11|-4582317743524655046|
| 12|   69662947042071568|
| 13| 7999406056282772309|
| 14|-2580036455355426926|
| 15| 6602252326686772978|
| 16|-3191246305972909392|
| 17| 5753536055347111814|
| 18|-8757599984427314983|
| 19| 5750159507242086581|
+---+--------------------+
only showing top 20 rows



In [21]:
val flightsMeta1 = spark.range(500).map(x => (x, scala.util.Random.nextLong))
  .withColumnRenamed("_1", "count").withColumnRenamed("_2", "randomData")

flightsMeta1 = [count: bigint, randomData: bigint]


[count: bigint, randomData: bigint]

In [22]:
flightsMeta1.show()

+-----+--------------------+
|count|          randomData|
+-----+--------------------+
|    0|-7211380657512360076|
|    1|-8254723276110045762|
|    2| 5949081505669866887|
|    3| 6616553353156927936|
|    4|-3391809214036079431|
|    5|-5030046179718028277|
|    6|-2583442328315128201|
|    7| 8583465300654768446|
|    8|-6585179668944574011|
|    9|-5689440713427345025|
|   10|-6497727397996804751|
|   11|-6248144817110949934|
|   12|-7962032803534689908|
|   13|-6840392436845325650|
|   14|-4977081244864977898|
|   15|-3058304738573760051|
|   16|-4441616574255991560|
|   17|  -77870753525344957|
|   18| -813165034660053706|
|   19| 8373022487123225042|
+-----+--------------------+
only showing top 20 rows



In [23]:
val flightsMeta = flightsMeta1.as[FlightMetadata]

flightsMeta = [count: bigint, randomData: bigint]


[count: bigint, randomData: bigint]

In [24]:
val flights2 = flights.joinWith(flightsMeta, flights.col("count") === flightsMeta.col("count"))

flights2 = [_1: struct<DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field>, _2: struct<count: bigint, randomData: bigint>]


[_1: struct<DEST_COUNTRY_NAME: string, ORIGIN_COUNTRY_NAME: string ... 1 more field>, _2: struct<count: bigint, randomData: bigint>]

In [25]:
flights2.show()

+--------------------+--------------------+
|                  _1|                  _2|
+--------------------+--------------------+
|[United States, R...|[1, -787775299908...|
|[United States, I...|[264, -1242102980...|
|[United States, I...|[69, -69849434228...|
|[Egypt, United St...|[24, -42334799634...|
|[Equatorial Guine...|[1, -787775299908...|
|[United States, S...|[25, -86329372422...|
|[United States, G...|[54, -26530291982...|
|[Costa Rica, Unit...|[477, -3792473526...|
|[Senegal, United ...|[29, -41509673180...|
|[United States, M...|[44, 811656018454...|
|[Guyana, United S...|[17, 125496987054...|
|[United States, S...|[53, -54674333291...|
|[Malta, United St...|[1, -787775299908...|
|[Bolivia, United ...|[46, 896996701485...|
|[Anguilla, United...|[21, -19589532279...|
|[Turks and Caicos...|[136, 21241592356...|
|[United States, A...|[2, 7062705438322...|
|[Saint Vincent an...|[1, -787775299908...|
|[Italy, United St...|[390, -4988177798...|
|[United States, R...|[156, 8431

In [26]:
flights2.take(10).foreach(println)

(Flight(United States,Romania,1),FlightMetadata(1,-3711323035090670621))
(Flight(United States,Ireland,264),FlightMetadata(264,3268759325966498349))
(Flight(United States,India,69),FlightMetadata(69,8899882629814114251))
(Flight(Egypt,United States,24),FlightMetadata(24,2734710184603806714))
(Flight(Equatorial Guinea,United States,1),FlightMetadata(1,-3711323035090670621))
(Flight(United States,Singapore,25),FlightMetadata(25,2330288661222830564))
(Flight(United States,Grenada,54),FlightMetadata(54,1520795149216698869))
(Flight(Costa Rica,United States,477),FlightMetadata(477,848690072306216806))
(Flight(Senegal,United States,29),FlightMetadata(29,-2619022139339652656))
(Flight(United States,Marshall Islands,44),FlightMetadata(44,-4682407921014983147))


In [27]:
flights2.selectExpr("_1.DEST_COUNTRY_NAME").show()

+--------------------+
|   DEST_COUNTRY_NAME|
+--------------------+
|       United States|
|       United States|
|       United States|
|               Egypt|
|   Equatorial Guinea|
|       United States|
|       United States|
|          Costa Rica|
|             Senegal|
|       United States|
|              Guyana|
|       United States|
|               Malta|
|             Bolivia|
|            Anguilla|
|Turks and Caicos ...|
|       United States|
|Saint Vincent and...|
|               Italy|
|       United States|
+--------------------+
only showing top 20 rows



In [28]:
flights2.selectExpr("_1.DEST_COUNTRY_NAME").take(4)

United States
United States
United States
Egypt


In [29]:
val flights3 = flights.join(flightsMeta, Seq("count"))

flights3 = [count: bigint, DEST_COUNTRY_NAME: string ... 2 more fields]


[count: bigint, DEST_COUNTRY_NAME: string ... 2 more fields]

In [30]:
flights3.take(4)

1,United States,Romania,-754198562442490722
264,United States,Ireland,4815993716481545711
69,United States,India,-8182651667234887706
24,Egypt,United States,-1831832207685697426


In [31]:
val flights4 = flights.join(flightsMeta.toDF(), Seq("count"))

flights4 = [count: bigint, DEST_COUNTRY_NAME: string ... 2 more fields]


[count: bigint, DEST_COUNTRY_NAME: string ... 2 more fields]

In [32]:
flights4.take(4)

1,United States,Romania,7226310089114737951
264,United States,Ireland,1631453143985871172
69,United States,India,-7242784686862571070
24,Egypt,United States,-585294014783270265


In [33]:
flights4.show()

+-----+--------------------+-------------------+--------------------+
|count|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|          randomData|
+-----+--------------------+-------------------+--------------------+
|    1|       United States|            Romania| 8573313110040646214|
|  264|       United States|            Ireland|-5683022886700030174|
|   69|       United States|              India|  884836151796415090|
|   24|               Egypt|      United States| 6720525618521304454|
|    1|   Equatorial Guinea|      United States| 8573313110040646214|
|   25|       United States|          Singapore| 2163182119373336278|
|   54|       United States|            Grenada|-3949040492800661041|
|  477|          Costa Rica|      United States| 1398907436853517000|
|   29|             Senegal|      United States|-5007689991307430076|
|   44|       United States|   Marshall Islands|-1669305580376728871|
|   17|              Guyana|      United States| 7318973232601248040|
|   53|       United

In [34]:
flights.join(flightsMeta.toDF(), "count").show()

+-----+--------------------+-------------------+--------------------+
|count|   DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|          randomData|
+-----+--------------------+-------------------+--------------------+
|    1|       United States|            Romania| 6236791750707601741|
|  264|       United States|            Ireland|-7099746024893441802|
|   69|       United States|              India|-1195023238738311532|
|   24|               Egypt|      United States|-1080671725318375702|
|    1|   Equatorial Guinea|      United States| 6236791750707601741|
|   25|       United States|          Singapore| 7665241361836221272|
|   54|       United States|            Grenada| 3249716655202423101|
|  477|          Costa Rica|      United States|-2812948112567178255|
|   29|             Senegal|      United States|-8895246107753437057|
|   44|       United States|   Marshall Islands| 6768718532109343926|
|   17|              Guyana|      United States|-4901922489851049716|
|   53|       United

In [35]:
flights.groupBy("DEST_COUNTRY_NAME").count().take(10)

Anguilla,1
Russia,1
Paraguay,1
Senegal,1
Sweden,1
Kiribati,1
Guyana,1
Philippines,1
Malaysia,1
Singapore,1


In [36]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).count().take(10)

[(Anguilla,1), (Russia,1), (Paraguay,1), (Senegal,1), (Sweden,1), (Kiribati,1), (Guyana,1), (Philippines,1), (Malaysia,1), (Singapore,1)]

In [37]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).count().explain

== Physical Plan ==
*(2) HashAggregate(keys=[value#432], functions=[count(1)])
+- Exchange hashpartitioning(value#432, 200)
   +- *(1) HashAggregate(keys=[value#432], functions=[partial_count(1)])
      +- *(1) Project [value#432]
         +- AppendColumns <function1>, newInstance(class $line33.$read$$iw$$iw$Flight), [staticinvoke(class org.apache.spark.unsafe.types.UTF8String, StringType, fromString, input[0, java.lang.String, true], true, false) AS value#432]
            +- InMemoryTableScan [DEST_COUNTRY_NAME#0, ORIGIN_COUNTRY_NAME#1, count#2L]
                  +- InMemoryRelation [DEST_COUNTRY_NAME#0, ORIGIN_COUNTRY_NAME#1, count#2L], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
                        +- *(1) FileScan parquet [DEST_COUNTRY_NAME#0,ORIGIN_COUNTRY_NAME#1,count#2L] Batched: true, Format: Parquet, Location: InMemoryFileIndex[hdfs://nn01.itversity.com:8020/user/kranthidr/dataSets/spark-guide/flight-data/..., PartitionFilters: [], PushedFilters: [],

In [38]:
def grpSum(countryName:String, values: Iterator[Flight]) = {
  values.dropWhile(_.count < 5).map(x => (countryName, x))
}

grpSum: (countryName: String, values: Iterator[Flight])Iterator[(String, Flight)]


In [39]:
flights.take(10)

[Flight(United States,Romania,1), Flight(United States,Ireland,264), Flight(United States,India,69), Flight(Egypt,United States,24), Flight(Equatorial Guinea,United States,1), Flight(United States,Singapore,25), Flight(United States,Grenada,54), Flight(Costa Rica,United States,477), Flight(Senegal,United States,29), Flight(United States,Marshall Islands,44)]

In [40]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).flatMapGroups(grpSum).take(10)

[(Anguilla,Flight(Anguilla,United States,21)), (Paraguay,Flight(Paraguay,United States,90)), (Russia,Flight(Russia,United States,152)), (Senegal,Flight(Senegal,United States,29)), (Sweden,Flight(Sweden,United States,65)), (Kiribati,Flight(Kiribati,United States,17)), (Guyana,Flight(Guyana,United States,17)), (Philippines,Flight(Philippines,United States,132)), (Singapore,Flight(Singapore,United States,25)), (Fiji,Flight(Fiji,United States,53))]

In [41]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).count().take(10)

[(Anguilla,1), (Russia,1), (Paraguay,1), (Senegal,1), (Sweden,1), (Kiribati,1), (Guyana,1), (Philippines,1), (Malaysia,1), (Singapore,1)]

In [42]:
flights.sort("DEST_COUNTRY_NAME").take(10)

[Flight(Afghanistan,United States,11), Flight(Angola,United States,14), Flight(Anguilla,United States,21), Flight(Antigua and Barbuda,United States,123), Flight(Argentina,United States,184), Flight(Aruba,United States,359), Flight(Australia,United States,290), Flight(Austria,United States,36), Flight(Azerbaijan,United States,1), Flight(Bahrain,United States,30)]

In [43]:
def grpSum2(f:Flight):Integer = {
  1
}

grpSum2: (f: Flight)Integer


In [44]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).mapValues(grpSum2).count().take(10)

[(Anguilla,1), (Russia,1), (Paraguay,1), (Senegal,1), (Sweden,1), (Kiribati,1), (Guyana,1), (Philippines,1), (Malaysia,1), (Singapore,1)]

In [45]:
def sum2(left:Flight, right:Flight) = {
 Flight(null, null, left.count + right.count)
}
def sum3(left:Flight, right:Flight) = {
 Flight(null, null, left.count - right.count)
}
def sum4(left:Flight, right:Flight) = {
 Flight(null, null, left.count * right.count)
}

sum2: (left: Flight, right: Flight)Flight
sum3: (left: Flight, right: Flight)Flight
sum4: (left: Flight, right: Flight)Flight


In [46]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).reduceGroups((l, r) => sum2(l, r)).take(10)

[(Anguilla,Flight(Anguilla,United States,21)), (Russia,Flight(Russia,United States,152)), (Paraguay,Flight(Paraguay,United States,90)), (Senegal,Flight(Senegal,United States,29)), (Sweden,Flight(Sweden,United States,65)), (Kiribati,Flight(Kiribati,United States,17)), (Guyana,Flight(Guyana,United States,17)), (Philippines,Flight(Philippines,United States,132)), (Malaysia,Flight(Malaysia,United States,1)), (Singapore,Flight(Singapore,United States,25))]

In [47]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).reduceGroups((l, r) => sum2(l, r)).take(10)

[(Anguilla,Flight(Anguilla,United States,21)), (Russia,Flight(Russia,United States,152)), (Paraguay,Flight(Paraguay,United States,90)), (Senegal,Flight(Senegal,United States,29)), (Sweden,Flight(Sweden,United States,65)), (Kiribati,Flight(Kiribati,United States,17)), (Guyana,Flight(Guyana,United States,17)), (Philippines,Flight(Philippines,United States,132)), (Malaysia,Flight(Malaysia,United States,1)), (Singapore,Flight(Singapore,United States,25))]

In [48]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).reduceGroups((l, r) => sum3(l, r)).take(10)

[(Anguilla,Flight(Anguilla,United States,21)), (Russia,Flight(Russia,United States,152)), (Paraguay,Flight(Paraguay,United States,90)), (Senegal,Flight(Senegal,United States,29)), (Sweden,Flight(Sweden,United States,65)), (Kiribati,Flight(Kiribati,United States,17)), (Guyana,Flight(Guyana,United States,17)), (Philippines,Flight(Philippines,United States,132)), (Malaysia,Flight(Malaysia,United States,1)), (Singapore,Flight(Singapore,United States,25))]

In [49]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).reduceGroups((l, r) => sum2(l, r)).count()

125

In [50]:
flights.groupByKey(x => x.DEST_COUNTRY_NAME).count().count()

125

In [51]:
flights.groupBy("DEST_COUNTRY_NAME").count().explain

== Physical Plan ==
*(2) HashAggregate(keys=[DEST_COUNTRY_NAME#0], functions=[count(1)])
+- Exchange hashpartitioning(DEST_COUNTRY_NAME#0, 200)
   +- *(1) HashAggregate(keys=[DEST_COUNTRY_NAME#0], functions=[partial_count(1)])
      +- InMemoryTableScan [DEST_COUNTRY_NAME#0]
            +- InMemoryRelation [DEST_COUNTRY_NAME#0, ORIGIN_COUNTRY_NAME#1, count#2L], true, 10000, StorageLevel(disk, memory, deserialized, 1 replicas)
                  +- *(1) FileScan parquet [DEST_COUNTRY_NAME#0,ORIGIN_COUNTRY_NAME#1,count#2L] Batched: true, Format: Parquet, Location: InMemoryFileIndex[hdfs://nn01.itversity.com:8020/user/kranthidr/dataSets/spark-guide/flight-data/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:bigint>
